<h1 align="center">Projet de DS51</h1>

<h2 align="center">Réalisation d'une ontologie et d'un modèle de reconnaissance d'images</h2>

Projet réalisé dans le cadre de l'UV DS51 (Modélisation de la connaissance) de l'UTBM, encadré par [Abderrahim Chariete](mailto:abderrahim.chariete@utbm.fr) enseignant-chercheur à l'UTBM, durant le semestre P23.

Réalisé par :
- [Léo Viguier](https://github.com/LeoViguier)
- [Thibault Chausson](https://github.com/thibault-chausson)

# I) Récupération des données

## Importation des données

Comme demandé dans le sujet, nous avons récupéré les données sur le site de la compétition [ImageNet Object Localization Challenge](https://www.kaggle.com/competitions/imagenet-object-localization-challenge) sur Kaggle. Nous avons ensuite extrait les données qui nous intéressent, à savoir les fichiers avec les identifiants suivant :
- n02114367 (loup)
- n01484850 (requin)
- n01614925 (aigle)
- n02133161 (ours)
- n01537544 (passerin indigo)
- n01443537 (poisson rouge)

## Récupération des noms d'animaux

### Récupération des identifiants des animaux sur Wikidata

Pour ce faire, nous allons récupérer la liste des différents animaux existant sur Terre grace à WikiData avec une request SPARQL :

```sparql
SELECT DISTINCT ?animal ?animalLabel WHERE {
  ?animal wdt:P31 wd:Q16521.
  ?animal rdfs:label ?animalLabel filter (lang(?animalLabel) = "en").
  #OPTIONAL {?animal wdt:P18 ?image.}
} LIMIT 1000000
```

Nous téléchargeons le fichier CSV pour les 1000000 premiers résultats et nous le sauvegardons dans le fichier `animals.csv`.

### Récupération des identifiants des animaux sur Kaggle

Sur Kaggle, nous récupérons le fichier [LOC_synset_mapping.txt](https://www.kaggle.com/competitions/imagenet-object-localization-challenge/data?select=LOC_synset_mapping.txt), qui contient la liste des identifiants des animaux avec leur nom.

La correspondance entre les 1000 synset id et leurs descriptions. Par exemple, la ligne 1 indique n01440764 tanche, Tinca tinca signifie qu'il s'agit de la classe 1, que l'identifiant synset est n01440764 et qu'elle contient le poisson tanche.

```txt
1	n01440764 tench, Tinca tinca
2	n01443537 goldfish, Carassius auratus
```

Commençons par charger les données dans un DataFrame Pandas. Nous allons utiliser le fichier `LOC_synset_mapping.txt` pour récupérer les identifiants des animaux et leur nom.

In [1]:
import pandas as pd

# Lecture du fichier texte
with open('LOC_synset_mapping.txt', 'r') as f:
    lignes = f.readlines()

# Création d'un DataFrame
df = pd.DataFrame({'ligne': lignes})

# Séparation de l'ID et du nom
df['id'] = df['ligne'].str.split(' ', n=1, expand=True)[0]
df['name'] = df['ligne'].str.split(' ', n=1, expand=True)[1].str.rstrip().str.split(',')

# Suppression de la colonne 'ligne'
df.drop(columns=['ligne'], inplace=True)

# Mise en minuscules des valeurs de la colonne 'name'
df['name'] = df['name'].apply(lambda x: [value.lower() for value in x])

# Affichage du DataFrame
df.head()

,id,name
0,n01440764,"[tench, tinca tinca]"
1,n01443537,"[goldfish, carassius auratus]"
2,n01484850,"[great white shark, white shark, man-eater, ..."
3,n01491361,"[tiger shark, galeocerdo cuvieri]"
4,n01494475,"[hammerhead, hammerhead shark]"


Nous venons de récupérer les identifiants et les noms des animaux, ou d'objet. Nous allons maintenant récupérer les identifiants des animaux dans le fichier `animals.csv` et les sauvegarder dans un nouveau fichier `animals_id.csv`. Commençons par lire le fichier `.csv`.

In [2]:
# Lecture du fichier CSV
df_animals = pd.read_csv('animals.csv')

# Conversion en minuscules de la colonne "animalLabel"
df_animals['animalLabel'] = df_animals['animalLabel'].str.lower()

# Affichage des premières lignes du DataFrame
df_animals.head()

,animal,animalLabel
0,http://www.wikidata.org/entity/Q140,lion
1,http://www.wikidata.org/entity/Q5113,birds
2,http://www.wikidata.org/entity/Q14334,wolverine
3,http://www.wikidata.org/entity/Q14272,geraniales
4,http://www.wikidata.org/entity/Q19064,agyrtidae


Comparons-les animalLabel avec les noms des animaux dans le fichier `LOC_synset_mapping.txt`. Pour cela, nous allons créer une fonction qui va comparer les deux listes et retourner l'identifiant de l'animal si il y a une correspondance.

In [3]:
# Définissons la liste des identifiants et des url Wikidata des animaux
ok = {"id": [], "name": []}

for id, name in zip(df['id'], df.name):
    for n in name:
        ok["id"].append(id.lower())
        ok["name"].append(n.lower())

okdf = pd.DataFrame(ok)
temps = pd.merge(df_animals, okdf, left_on='animalLabel', right_on='name')

temps = temps.dropna()

animals = temps.groupby(['animal', 'id']).first().reset_index()
animals = animals.drop('name', axis=1)  # Supprimer la colonne 'name'
animals.to_csv('animals_id.csv', index=False)  # Sauvegarder le fichier

Ajouter à `animals_id.csv` les animaux que nous devons implémenter, s'ils ne sont pas dans le fichier. Si nous devons ajouter un animal, il n'y a aucun intérêt à le faire manuellement, nous allons donc le faire automatiquement. Ainsi, nous n'allons pas rechercher ses informations dans WikiData, car il est compliqué de déterminer automatiquement si un wolf est un animal ou un personnage de fiction.

In [4]:
# Lien entre les noms des dossiers et les noms des animaux
correspondance_dossier_animaux = {'n02114367': 'wolf', 'n01484850': 'shark', 'n01614925': 'eagle', 'n02133161': 'bear',
                                  'n01537544': 'Indigo Bunting', 'n01443537': 'goldfish'}

df_id_animaux = pd.read_csv('animals_id.csv')

# Ajout des animaux manquants au DataFrame
for id, name in correspondance_dossier_animaux.items():
    if id not in df_id_animaux['id'].values:
        df_id_animaux = df_id_animaux.append({'animalLabel': name, 'id': id}, ignore_index=True)

df_id_animaux.to_csv('animals_id.csv', index=False)

/var/folders/c6/dvmqz81j59g3qs_b7r4gm6640000gn/T/ipykernel_1643/3232855127.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_id_animaux = df_id_animaux.append({'animalLabel': name, 'id': id}, ignore_index=True)
/var/folders/c6/dvmqz81j59g3qs_b7r4gm6640000gn/T/ipykernel_1643/3232855127.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_id_animaux = df_id_animaux.append({'animalLabel': name, 'id': id}, ignore_index=True)


## Récupération des données depuis une base de données : WikiData

Pour ce faire, nous utiliserons la librairie `wikibaseintegrator` qui permet d'interroger la base de données de WikiData.
Nous voulons récupérer l'ensemble des animaux de la base de données, pour ce faire, nous utiliserons : WikibaseIntegrator wbi.item.get(entity_id='Q729').

In [5]:
from wikibaseintegrator import wbi_login, WikibaseIntegrator
from wikibaseintegrator.wbi_config import config as wbi_config

wbi_config['USER_AGENT'] = 'MyWikibaseBot/1.0 (https://www.wikidata.org/wiki/User:MyUsername)'

# login object
login_instance = wbi_login.Clientlogin(user='Arti0890', password='purtyx-fuwvi4-rYmgor')

wbi = WikibaseIntegrator(login=login_instance)

item = wbi.item.get(entity_id='Q729')

# to check successful installation and retrieval of the data, you can print the json representation of the item
print(item.get_json())

{'labels': {'pl': {'language': 'pl', 'value': 'zwierzęta'}, 'eo': {'language': 'eo', 'value': 'animalo'}, 'en': {'language': 'en', 'value': 'animal'}, 'nl': {'language': 'nl', 'value': 'dier'}, 'it': {'language': 'it', 'value': 'animale'}, 'fr': {'language': 'fr', 'value': 'animal'}, 'be-tarask': {'language': 'be-tarask', 'value': 'жывёлы'}, 'nan': {'language': 'nan', 'value': 'Tōng-bu̍t'}, 'es': {'language': 'es', 'value': 'Animalia'}, 'en-ca': {'language': 'en-ca', 'value': 'Animal'}, 'en-gb': {'language': 'en-gb', 'value': 'animal'}, 'ace': {'language': 'ace', 'value': 'Binatang'}, 'af': {'language': 'af', 'value': 'dier'}, 'am': {'language': 'am', 'value': 'እንስሳ'}, 'an': {'language': 'an', 'value': 'Animalia'}, 'ar': {'language': 'ar', 'value': 'حيوان'}, 'arc': {'language': 'arc', 'value': 'ܚܝܘܬܐ'}, 'ast': {'language': 'ast', 'value': 'animal'}, 'ay': {'language': 'ay', 'value': 'Uywa'}, 'az': {'language': 'az', 'value': 'heyvan'}, 'ba': {'language': 'ba', 'value': 'Хайуандар'}, 'b

Ou, grâce à une requête SPARQL directement, méthode utilisée plus tard pour interroger les sous-classes des animaux.

In [6]:
from wikibaseintegrator import wbi_helpers

# Requête SPARQL pour récupérer les animaux
query = """
SELECT DISTINCT ?animal ?animalLabel WHERE {
  ?animal wdt:P31 wd:Q729.
  ?animal rdfs:label ?animalLabel filter (lang(?animalLabel) = "en").
  #OPTIONAL {?animal wdt:P18 ?image.}
} LIMIT 100
"""

# Récupérer les entités correspondant à la requête SPARQL
data = wbi_helpers.execute_sparql_query(query)
print(data)

{'head': {'vars': ['animal', 'animalLabel']}, 'results': {'bindings': [{'animal': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q117052967'}, 'animalLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Gold Tony'}}, {'animal': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q117100618'}, 'animalLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'wu song tiger'}}, {'animal': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q117812772'}, 'animalLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'rys-wyrm'}}, {'animal': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q118576045'}, 'animalLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'phocoena'}}]}}


## Récupération des sous-classes des animaux

Nous allons maintenant récupérer les sous-classes des animaux. Pour cela, nous allons utiliser la requête SPARQL suivante :
```SPARQL
    SELECT ?animal ?animalLabel ?subclass ?subclassLabel ?subclass2 ?subclass2Label
    WHERE
    {
      VALUES ?animal {wd:%s}
      ?animal wdt:P279 ?subclass .
      OPTIONAL { ?subclass wdt:P279 ?subclass2 . }

      SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
    }
```
Nous avons récupéré la propriété `subclass of`, au lieu de `instant of`, car cette deuxième n'était pas toujours présente dans la liste des animaux fournis, contrairement à la première.

Nous utilisons la bibliothèque `wikibaseintegrator` pour interroger la base de données de WikiData, ceci nous permet de ne pas nous faire bloquer par l'API WikiData si nous faisons trop de requêtes.

In [7]:
with open('animals_id.csv', 'r') as f:
    next(f)  # Ignorer la première ligne
    wd_list = [[line.split('/')[-1].strip().split(',')[0], line.strip().split(',')[2], line.strip().split(',')[1]] for
               line in f.readlines()]

resultat = {"id_wiki": [], "animalLabel": [], "subclassLabel": [], "subclass2Label": [], "id": []}

for wd in wd_list:
    # Construction de la requête SPARQL
    query = """
    SELECT ?animal ?animalLabel ?subclass ?subclassLabel ?subclass2 ?subclass2Label
    WHERE
    {
      VALUES ?animal {wd:%s}
      ?animal wdt:P279 ?subclass .
      OPTIONAL { ?subclass wdt:P279 ?subclass2 . }

      SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
    }
    """ % wd[0]

    # Execution de la requête SPARQL
    data = wbi_helpers.execute_sparql_query(query)

    # Ajout des id_wiki
    resultat["id_wiki"].append(wd[0])

    if data['results']['bindings']:
        print('------')
        print('Animal:', data['results']['bindings'][0]['animalLabel']['value'])
        print('Subclass:', data['results']['bindings'][0]['subclassLabel']['value'])
        resultat["animalLabel"].append(wd[1].lower())
        resultat["subclassLabel"].append(data['results']['bindings'][0]['subclassLabel']['value'].lower())
        if data['results']['bindings'][0]['animalLabel']['value']:
            print('Subclass of Subclass:', data['results']['bindings'][0]['subclass2Label']['value'])
            resultat["subclass2Label"].append(data['results']['bindings'][0]['subclass2Label']['value'].lower())
        else:
            resultat["subclass2Label"].append(0)
        resultat["id"].append(wd[2])
        print('------')

    else:
        print('No results for', wd[0])
        resultat["animalLabel"].append(wd[1].lower())
        resultat["subclassLabel"].append(0)
        resultat["subclass2Label"].append(0)
        resultat["id"].append(wd[2])

#enregistrement en csv
df = pd.DataFrame(resultat)
df.to_csv('animals_id_name_subClass_sub2Class.csv', index=False)

No results for Q1049642
No results for Q1144302
No results for Q1149509
No results for Q1153386
No results for Q123141
No results for Q127216
No results for Q129026
No results for Q129544
No results for Q1306991
No results for Q131044
No results for Q13158
No results for Q132576
No results for Q132585
No results for Q134015
------
Animal: lion
Subclass: big cat
Subclass of Subclass: Felidae
------
No results for Q145016
No results for Q1522299
No results for Q15343
No results for Q155878
No results for Q159404
No results for Q17700
No results for Q177932
No results for Q18099
No results for Q1813704
No results for Q181871
------
Animal: King Penguin
Subclass: penguin
Subclass of Subclass: penguin
------
------
Animal: African elephant
Subclass: elephant
Subclass of Subclass: mammal
------
No results for Q185167
No results for Q185939
No results for Q1861297
No results for Q188877
No results for Q189609
No results for Q19058
No results for Q19125
No results for Q192029
No results for Q1

Nous venons de récupérer l'ensemble des informations dont nous avons besoin pour notre projet, c'est-à-dire un fichier csv contenant les informations suivantes :
- id_wiki : l'id de l'animal dans la base de données de WikiData
- animalLabel : le nom de l'animal
- subclassLabel : le nom de la sous-classe de l'animal
- subclass2Label : le nom de la sous-classe de la sous-classe de l'animal
- id : l'id de l'animal dans la base de données d'ImageNet, à savoir le dossier où se trouvent les images de l'animal

Nous allons maintenant utiliser ces informations pour entrainer notre modèle. La suite du projet se trouve dans le notebook [2-1_data_preparation.ipynb](./2-1_data_preparation.ipynb).